In [2]:
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy



In [3]:
environment_name = "CartPole-v0"
env = gym.make(environment_name)



In [3]:
episodes = 5
for episode in range(1,episodes+1):
    state=env.reset()
    done=False
    score = 0
    
    while not done:
        env.render()
        action=env.action_space.sample()
        n_state,reward,done,info = env.step(action)
        score+=reward
    print('Episode:{} Score;{}'.format(episode,score))
env.close()

Episode:1 Score;29.0
Episode:2 Score;21.0
Episode:3 Score;26.0
Episode:4 Score;30.0
Episode:5 Score;35.0


In [4]:
env.close()

In [4]:
import os
log_path = os.path.join('Training', 'logs')

In [6]:
log_path

'Training/logs'

In [5]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)


Using cuda device


In [8]:
PPO??


In [9]:
model.learn(total_timesteps=20000)

Logging to Training/logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 756  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 741         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012010581 |
|    clip_fraction        | 0.112       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00245     |
|    learning_rate        | 0.0003      |
|    loss                 | 8.73        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0177     |
|    value_loss           | 49.9        |
-----------------------------------------
---

In [7]:
PPO_Path = os.path.join('Training','Saved Models','PPo_Model_cartpole')

In [11]:
model.save(PPO_Path)

/home/vedant/miniconda3/envs/tensorflow/lib/python3.7/site-packages/stable_baselines3/common/save_util.py:276: UserWarning: Path 'Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [12]:
del model

In [8]:
model = PPO.load(PPO_Path,env=env)

In [14]:
model.learn(total_timesteps=1000)

Logging to Training/logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 1099 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------


In [15]:
evaluate_policy(model,env,n_eval_episodes=10,render=True)

/home/vedant/miniconda3/envs/tensorflow/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(200.0, 0.0)

In [16]:
env.close()

In [17]:
episodes = 5
for episode in range(1,episodes+1):
    obs=env.reset()
    done=False
    score = 0
    
    while not done:
        env.render()
        action,_=model.predict(obs)
        obs,reward,done,info = env.step(action)
        score+=reward
    print('Episode:{} Score;{}'.format(episode,score))
env.close()

Episode:1 Score;[200.]
Episode:2 Score;[200.]
Episode:3 Score;[200.]
Episode:4 Score;[200.]
Episode:5 Score;[200.]


In [9]:
training_log_path=os.path.join(log_path,'PPO_1')

In [20]:
training_log_path

'Training/logs/PPO_1'

In [ ]:
!tensorboard --logdir={training_log_path}


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.5.0 at http://localhost:6006/ (Press CTRL+C to quit)


In [10]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [11]:
save_path=os.path.join('Training','Saved Models')

In [12]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)


In [13]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

Using cuda device


In [14]:
model.learn(total_timesteps=20000,callback=eval_callback)

Logging to Training/logs/PPO_3
-----------------------------
| time/              |      |
|    fps             | 551  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 585          |
|    iterations           | 2            |
|    time_elapsed         | 6            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0036954272 |
|    clip_fraction        | 0.0837       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.687       |
|    explained_variance   | 0.00133      |
|    learning_rate        | 0.0003       |
|    loss                 | 7.55         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0125      |
|    value_loss           | 53.2         |
----------------------------

/home/vedant/miniconda3/envs/tensorflow/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Eval num_timesteps=10000, episode_reward=194.80 +/- 10.40
Episode length: 194.80 +/- 10.40
New best mean reward!
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 195         |
|    mean_reward          | 195         |
| time/                   |             |
|    fps                  | 598         |
|    iterations           | 5           |
|    time_elapsed         | 17          |
|    total_timesteps      | 10240       |
| train/                  |             |
|    approx_kl            | 0.011146549 |
|    clip_fraction        | 0.0874      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.607      |
|    explained_variance   | 0.284       |
|    learning_rate        | 0.0003      |
|    loss                 | 25.6        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0189     |
|    value_loss           | 65.5        |
-----------------------------------------
-----

In [17]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [18]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})


Using cuda device


In [19]:
model.learn(total_timesteps=20000,callback=eval_callback)

----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 200        |
|    mean_reward          | 200        |
| time/                   |            |
|    fps                  | 947        |
|    iterations           | 1          |
|    time_elapsed         | 2          |
|    total_timesteps      | 2048       |
| train/                  |            |
|    approx_kl            | 0.00551892 |
|    clip_fraction        | 0.0202     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.544     |
|    explained_variance   | 0.355      |
|    learning_rate        | 0.0003     |
|    loss                 | 16.2       |
|    n_updates            | 90         |
|    policy_gradient_loss | -0.00217   |
|    value_loss           | 94.5       |
----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 669         |
|    iteratio

In [21]:
from stable_baselines3 import DQN

In [24]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)


Using cuda device


In [25]:
model.learn(total_timesteps=20000,callback=eval_callback)

Logging to Training/logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration rate | 0.955    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3613     |
|    time_elapsed     | 0        |
|    total timesteps  | 95       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.922    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 4599     |
|    time_elapsed     | 0        |
|    total timesteps  | 165      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.876    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 5818     |
|    time_elapsed     | 0        |
|    total timesteps  | 262      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 9317     |
|    time_elapsed     | 0        |
|    total timesteps  | 2663     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 9367     |
|    time_elapsed     | 0        |
|    total timesteps  | 2739     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 9378     |
|    time_elapsed     | 0        |
|    total timesteps  | 2823     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 9335     |
|    time_elapsed     | 0        |
|    total timesteps  | 4980     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 9318     |
|    time_elapsed     | 0        |
|    total timesteps  | 5045     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 9324     |
|    time_elapsed     | 0        |
|    total timesteps  | 5151     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 9533     |
|    time_elapsed     | 0        |
|    total timesteps  | 7315     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 9558     |
|    time_elapsed     | 0        |
|    total timesteps  | 7417     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 9572     |
|    time_elapsed     | 0        |
|    total timesteps  | 7539     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 428      |
|    fps              | 9400     |
|    time_elapsed     | 1        |
|    total timesteps  | 9654     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 9396     |
|    time_elapsed     | 1        |
|    total timesteps  | 9777     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 9388     |
|    time_elapsed     | 1        |
|    total timesteps  | 9878     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 9334     |
|    time_elapsed     | 1        |
|    total timesteps  | 12115    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 9328     |
|    time_elapsed     | 1        |
|    total timesteps  | 12173    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 544      |
|    fps              | 9331     |
|    time_elapsed     | 1        |
|    total timesteps  | 12233    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 9355     |
|    time_elapsed     | 1        |
|    total timesteps  | 14463    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 9356     |
|    time_elapsed     | 1        |
|    total timesteps  | 14529    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 652      |
|    fps              | 9361     |
|    time_elapsed     | 1        |
|    total timesteps  | 14582    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 9382     |
|    time_elapsed     | 1        |
|    total timesteps  | 16671    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 9388     |
|    time_elapsed     | 1        |
|    total timesteps  | 16779    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 760      |
|    fps              | 9395     |
|    time_elapsed     | 1        |
|    total timesteps  | 16867    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 9450     |
|    time_elapsed     | 2        |
|    total timesteps  | 19207    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 9449     |
|    time_elapsed     | 2        |
|    total timesteps  | 19264    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 868      |
|    fps              | 9454     |
|    time_elapsed     | 2        |
|    total timesteps  | 19358    |
----------------------------------
----------------------------------
| rollout/          

In [27]:
DQN_Path=os.path.join('Training','Saved Models','DQN_Model_cartpole')

In [28]:
model.save(DQN_Path)